# Launch post-run analysis jobs
Code to create a batch script for launching jobs on cori
Sep 1, 2020

In [1]:
import os
import glob
import time
import subprocess as sp
import numpy as np

In [2]:
curr_dir=os.getcwd()
print(curr_dir)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/4_basic_3d_GAN


In [4]:
val_files={'32':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_100k/norm_1_train_val.npy',
          '512':''
}

In [11]:
results_loc='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/'

In [6]:
ls /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/

20210111_104029_3d_/


In [6]:
# glob.glob(results_loc+'*')

In [7]:
ip_folder='20210111_104029_3d_'

In [12]:
### Default dictionary
img_size=32
dict_pars={'job_name':'analysis_3d', 'queue':'debug',
           'time':'00:30:00'}

dict_pars['val_file']=val_files[str(img_size)]
dict_pars['ip_folder']=results_loc+ip_folder

print(dict_pars)

{'job_name': 'analysis_3d', 'queue': 'debug', 'time': '00:30:00', 'val_file': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_100k/norm_1_train_val.npy', 'ip_folder': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210111_104029_3d_'}


In [13]:
assert os.path.isdir(dict_pars['ip_folder'])

In [19]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos={queue}
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time={time}
#SBATCH --job-name={job_name}

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

conda activate v3
code_dir='/global/homes/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/4_basic_3d_GAN/1_main_code'

python $code_dir/post_analysis_pandas.py -f {ip_folder}  -v {val_file}
conda deactivate
echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [20]:
# bash_strg

In [24]:
### Write bash file
staging_loc='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/'
fname='batch_analysis_3d.sh'
filename=staging_loc+fname
print(filename)
with open (filename,'w') as f:
    f.write(bash_strg)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/batch_analysis_3d.sh


In [25]:
### Move to staging locations in project space:
os.chdir(staging_loc)

In [26]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=debug
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time=00:30:00
#SBATCH --job-name=analysis_3d

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

conda activate v3
code_dir='/global/homes/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/4_basic_3d_GAN/1_main_code'

python $code_dir/post_analysis_pandas.py -f /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210111_104029_3d_  -v /global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_100k/norm_1_train_val.npy
conda deactivate
echo "--end date" `date` `date +%s`


### Submit jobs to cori queue

In [14]:
%%bash -s "$filename" ## Use python variable in bash
sbatch $1

Submitted batch job 37312892
